In [1]:
from praatio import textgrid
from praatio.utilities.constants import Interval
from alignedTextGrid.sequences.sequences import SequenceInterval, Top, Bottom
from alignedTextGrid.sequences.tiers import SequenceTier, RelatedTiers

In [2]:
spritely = textgrid.openTextgrid(
    fnFullPath="spritely.TextGrid",
    includeEmptyIntervals=True
)

In [3]:
prosodic_tnames = spritely.tierNames
prosodic_tnames

('PrWord', 'Foot', 'Syl', 'OnsetRime', 'SylPart', 'Phone')

In [4]:
class PrWord(SequenceInterval):
    def __init__(self, Interval):
        super().__init__(Interval)
        self.set_superset_class(Top)
        self.set_subset_class(Foot)

class Foot(SequenceInterval):
    def __init__(self, Interval):
        super().__init__(Interval)
        self.set_superset_class(PrWord)
        self.set_subset_class(Syl)

class Syl(SequenceInterval):
    def __init__(self, Interval):
        super().__init__(Interval)
        self.set_superset_class(Foot)
        self.set_subset_class(OnsetRime)

class OnsetRime(SequenceInterval):
    def __init__(self, Interval):
        super().__init__(Interval)
        self.set_superset_class(Syl)
        self.set_subset_class(SylPart)

class SylPart(SequenceInterval):
    def __init__(self, Interval):
        super().__init__(Interval)
        self.set_superset_class(OnsetRime)
        self.set_subset_class(Phone)

class Phone(SequenceInterval):
    def __init__(self, Interval):
        super().__init__(Interval)
        self.set_superset_class(SylPart)
        self.set_subset_class(Bottom)


In [5]:
PrTier = SequenceTier(spritely.getTier("PrWord"), entry_class=PrWord)
FootTier = SequenceTier(spritely.getTier("Foot"), entry_class=Foot)
SylTier = SequenceTier(spritely.getTier("Syl"), entry_class=Syl)
OnsetRimeTier = SequenceTier(spritely.getTier("OnsetRime"), entry_class=OnsetRime)
SylPartTier = SequenceTier(spritely.getTier("SylPart"), entry_class=SylPart)
PhoneTier = SequenceTier(spritely.getTier("Phone"), entry_class=Phone)

In [6]:
stacked_tiers = RelatedTiers([PrTier, FootTier, SylTier, OnsetRimeTier, SylPartTier, PhoneTier])

In [7]:
for i in stacked_tiers[5]:
    print(i.label)


S
P
R
AY
T
L
IY



In [8]:
# Info about a sequence
print(stacked_tiers[5][3])
print(stacked_tiers[5][3].label)
print(stacked_tiers[5][3].start)
print(stacked_tiers[5][3].tier_index)

Class Phone, label: R, .superset_class: SylPart, .super_instance: O, .subset_class: Bottom
R
1.4605902223946439
3


In [9]:
# precedence relationsips
print(stacked_tiers[5][3].fol.label)
print(stacked_tiers[5][3].prev.label)

#
P


In [16]:
# hierarchical relationships\
print(stacked_tiers[5][3].super_instance) # O = onset
print(stacked_tiers[5][3].subset_list)

Class SylPart, label: O, .superset_class: OnsetRime, .super_instance: O, .subset_class: Phone, .subset_list: ['S', 'P', 'R']
[]


In [15]:
# up then down
for i in stacked_tiers[5][3].super_instance.subset_list:
    print(i)

Class Phone, label: S, .superset_class: SylPart, .super_instance: O, .subset_class: Bottom
Class Phone, label: P, .superset_class: SylPart, .super_instance: O, .subset_class: Bottom
Class Phone, label: R, .superset_class: SylPart, .super_instance: O, .subset_class: Bottom


You have to travel further to from the onset to the nucelus than from the nucleus to the coda

In [14]:
(stacked_tiers[5][3]   # onset phone
    .super_instance    # (onset)
    .super_instance    # Onset 
    .fol               # Rhyme
    .subset_list[0]    # Nucleus
    .subset_list[0]    # Nucleus phone
    .label
)

'AY'

In [13]:
(stacked_tiers[5][4]   # Nucleus phone
    .super_instance    # Nucleus
    .fol               # Coda
    .subset_list[0]    # Coda phone
    .label
)

'T'